Buid a basic chatbot with LangGraph(Graph API)

In [ ]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
from langchain_groq import ChatGroq
from langchain.chat_models import init_chat_model

llm=ChatGroq(model="llama3-8b-8192")

In [ ]:
# llm=init_chat_model("groq:llama3-8b-8192")

In [ ]:
llm

In [ ]:
def chatbot(state: State):
    return {"messages": [llm.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(State)
graph_builder.add_node("llmchatbot",chatbot)

graph_builder.add_edge(START, "llmchatbot")
graph_builder.add_edge("llmchatbot", END)

graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({"messages": "Hi"})

In [ ]:
response["messages"][-1].content

In [ ]:
for event in graph.stream({"messages": "Hi, How are you?"}):
    for value in event.values():
        print(value["messages"][-1].content)

ChatBot with tools

In [ ]:
from langchain_tavily import TavilySearch

tool=TavilySearch(max_results=2)

In [ ]:
def multiply(a: int, b: int) -> int:
    """
    Multiply two integers and return the result.

    Args:
        a (int): The first integer.
        b (int): The second integer.

    Returns:
        int: The product of a and b.
    """ 
    return a * b

In [ ]:
tools = [tool, multiply]

In [ ]:
llm_with_tools=llm.bind_tools(tools)

In [ ]:
llm_with_tools

In [ ]:
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition

def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder=StateGraph(State)
builder.add_node("tool_calling_llm",tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)

graph=builder.compile()

In [ ]:
from IPython.display import display, Image

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response=graph.invoke({"messages": "what is the recent ai news and then multiply 2 and 3?"})

for m in response["messages"]:
    m.pretty_print()

Adding memory in Agentic